In [2]:
import pandas as pd  #importing pandas  
import glob #importing glob to deal with iterations within a directory
import datetime #importing date time for date time manipulation

# Creating a collection of data frame to store various sites in diffrent frames
dataframe_collection = {}

# Creating a temporary data frame to store temp values for processing
all_data = pd.DataFrame()

#Setting the date range using pandas
date1 = '2017-08'
date2 = '2020-03'
klist= pd.date_range('2018-01','2020-03',freq='M').strftime("%Y-%m").tolist()

#using 3 Loops, k loop to manage the date range (change if required), j loop is to manage the sites since (we chose 5 sites ranges from 1-6), the third loop is to iterate through diffrent files inside a given directory
for k in range(0,26):
    for j in range(1,6):
        i=1
        for f in glob.glob("../python_scripts/NeonDataNEON.D08.TALL.DP1.00094.001.%s.basic.*.zip/NEON.D08.TALL.DP1.00094.001.00%d.*.030.*"%(klist[k],j)):
#reading a single file and deleting and formatting the data based on our requirement
            df = pd.read_csv(f)
            del df['VSWCMinimum']
            del df['VSWCMaximum']
            del df['VSWCNumPts']
            del df['VSWCExpUncert']
            del df['VSWCStdErMean']
            del df['VSICMinimum']
            del df['VSICMaximum']
            del df['VSICNumPts']
            del df['VSICExpUncert']
            del df['VSICStdErMean']
            del df['VSWCVariance']
            del df['VSICVariance']
            df['startDateTime']= pd.to_datetime(df['startDateTime'])
            
#resampling our existing data by averaging our data to daily data index is start date time 
            df=df.resample('d', on='startDateTime').mean()
 #Checking whether this is the first file for a given site which is being processes if first add to the orginal data frame since it is empty      
            if k==0:
                if i==1:
                    dataframe_collection[j] = pd.DataFrame(index = df.index)
                    dataframe_collection[j]['Level1VSWCMean'] = df['VSWCMean']
                    dataframe_collection[j]['Level1VSWCFinalQF'] = df['VSWCFinalQF']
                    dataframe_collection[j]['Level1VSICMean'] = df['VSICMean']
                    dataframe_collection[j]['Level1VSICFinalQF'] = df['VSICFinalQF'] 

                else:
                    dataframe_collection[j]['Level%dVSWCMean'%i] = df['VSWCMean']
                    dataframe_collection[j]['Level%dVSWCFinalQF' %i] = df['VSWCFinalQF']
                    dataframe_collection[j]['Level%dVSICMean' %i] = df['VSICMean']
                    dataframe_collection[j]['Level%dVSICFinalQF'%i] = df['VSICFinalQF']
#if it is not the first file for a given site then we store the processed data in a temporary data frame and append it after processing
            else:
                if i==1:
                    all_data = pd.DataFrame(index = df.index)
                    all_data['Level1VSWCMean'] = df['VSWCMean']
                    all_data['Level1VSWCFinalQF'] = df['VSWCFinalQF']
                    all_data['Level1VSICMean'] = df['VSICMean']
                    all_data['Level1VSICFinalQF'] = df['VSICFinalQF'] 

                else:
                    all_data['Level%dVSWCMean'%i] = df['VSWCMean']
                    all_data['Level%dVSWCFinalQF' %i] = df['VSWCFinalQF']
                    all_data['Level%dVSICMean' %i] = df['VSICMean']
                    all_data['Level%dVSICFinalQF'%i] = df['VSICFinalQF']
            i=i+1
#appending the data frame
            dataframe_collection[j]=dataframe_collection[j].append(all_data)

# filling empty or missing values with -999
dataframe_collection[1] = dataframe_collection[1].fillna(-999)
dataframe_collection[2] = dataframe_collection[2].fillna(-999)
dataframe_collection[3] = dataframe_collection[3].fillna(-999)
dataframe_collection[4] = dataframe_collection[4].fillna(-999)
dataframe_collection[5] = dataframe_collection[5].fillna(-999)

#stripping our timezone from datetime so that we can get it into a format which can be stored in excel)
dataframe_collection[1].index=dataframe_collection[1].index.astype(str).str[:-6]
dataframe_collection[2].index=dataframe_collection[2].index.astype(str).str[:-6]
dataframe_collection[3].index=dataframe_collection[3].index.astype(str).str[:-6]
dataframe_collection[4].index=dataframe_collection[4].index.astype(str).str[:-6]
dataframe_collection[5].index=dataframe_collection[5].index.astype(str).str[:-6]
#writing the cleaned data to a new excel file where 5 sites are stored in 5 diffrent sheets        
fname = 'FinalDataTALL2018-2020.xlsx'
writer = pd.ExcelWriter(fname,engine = 'xlsxwriter',options = {'remove_timezone': True})
dataframe_collection[1].to_excel(writer, "Site1")
dataframe_collection[2].to_excel(writer, "Site2")
dataframe_collection[3].to_excel(writer, "Site3")
dataframe_collection[4].to_excel(writer, "Site4")
dataframe_collection[5].to_excel(writer, "Site5")
## more dataframes goes here
writer.save()
